### Reading parameters
1) Complete a .env file with the model, endpoint and api key that you get from MAAS: https://maas.apps.prod.rhoai.rh-aiservices-bu.com/admin/applications/5585?service_id=10

#### For instance:
MAAS_GRANITE_38="{\"model\": \"granite-3-8b-instruct\", \"apikey\": \"xxxxx\", \"endpoint\": \"https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\"}"





#todo parameters for langchain

In [1]:
import requests
import urllib3
import numpy as np
import json
import os
from dotenv import load_dotenv



In [2]:
def read_parameter_dict(dict_name):
    load_dotenv()
    return(json.loads(os.getenv(dict_name)))


In [3]:
def read_model_parameters(dict_name):
    load_dotenv()
    par_dict=json.loads(os.getenv(dict_name))
    model=par_dict['model']
    apikey=par_dict['apikey']
    endpoint=par_dict['endpoint']
    return(model, apikey, endpoint)


In [4]:
def request_inference(model_name,endpoint, apikey, prompt,max_token=15, temperature=0):
    # print(f"Model {model_name} Endpoint {endpoint} {prompt}")
    completion = requests.post(
    url=endpoint+'/v1/completions',
    json={
      "model": model_name,
      "prompt": prompt,
      "max_tokens": max_token,
      "temperature": temperature
    },
    headers={'Authorization': 'Bearer '+apikey}).json()
    return completion

In [6]:
from langchain_community.llms import VLLMOpenAI
def langchain_inference(model,endpoint, apikey, prompt):
    llm = VLLMOpenAI(
        openai_api_key=apikey,
        openai_api_base=endpoint+"/v1",
        model_name=model,
        model_kwargs={"stop": ["."]},
    )
    return(llm.invoke(prompt))

## Testing inference granite 3.1-8b-instruct
Model: granite-3-8b-instruct

Hugging Face: https://huggingface.co/ibm-granite/granite-3.1-8b-instruct








In [7]:
granite_38=read_parameter_dict("MAAS_GRANITE_38")
granite_38

{'model': 'granite-3-8b-instruct',
 'apikey': '157356977debf113cb5275dccf0b3f75',
 'endpoint': 'https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443'}

In [8]:
response=request_inference(granite_38['model'],granite_38['endpoint'],granite_38['apikey'], "San Francisco es",50)
response['choices'][0]['text']

' una ciudad que se caracteriza por su diversidad cultural, su vida nocturna vibrante y su impresionante arquitectura. Es una ciudad que ofrece una gran variedad de activ'

In [10]:
response=langchain_inference(granite_38['model'],granite_38['endpoint'],granite_38['apikey'], "Roma es")
response

' una de las cunas de la civilización occidental y durante millones de años ha sido una de las ciudades más importantes del mundo'

### Testing DeepSeek-R1
https://huggingface.co/deepseek-ai/DeepSeek-R1



In [ ]:
deepseek_r1=read_parameter_dict("DEEPSEEK_R1")
deepseek_r1

In [10]:
model=deepseek_r1['model']
endpoint=deepseek_r1['endpoint']
apikey=deepseek_r1['apikey']
prompt="how many rectangles of 10cm x 13 cm enter in a square meter?"
max_tokens=1000

In [11]:
response=request_inference(model, endpoint,apikey, prompt, 500)


In [ ]:
response['choices'][0]['text']

### Testing Granite-Embedding-278m-multilingual
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [22]:
emb_model, emb_apikey, emb_endpoint=read_model_parameters("GRANITEEMBEDDING")

In [ ]:
print(f"{emb_endpoint}  {emb_apikey} {emb_model}")

In [ ]:
test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]

embeddings = requests.post(
    url=emb_endpoint+'/v1/embeddings',
    json={"input": test_strings, "model": emb_model},
    headers={'Authorization': 'Bearer '+emb_apikey}
).json()

print(embeddings)

### Testing Nomic embed-text-v1-5
https://www.nomic.ai/blog/posts/nomic-embed-text-v1



In [16]:
emb_model, emb_apikey, emb_endpoint=read_model_parameters("NAOEMIC")

In [17]:
from langchain_openai import OpenAIEmbeddings


embeddings_model = OpenAIEmbeddings(openai_api_base=emb_endpoint, openai_api_key=emb_apikey, model=emb_model)

test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]



In [ ]:
print(f"{emb_endpoint}  {emb_apikey} {emb_model}")

In [ ]:
embeddings = embeddings_model.embed_documents(test_strings)

print(embeddings)

In [ ]:
from langchain_openai import OpenAIEmbeddings

API_URL = "https://nomic-embed-text-v1-5-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443"
API_KEY = emb_apikey

embeddings_model = OpenAIEmbeddings(openai_api_base=API_URL, openai_api_key=API_KEY, model=emb_model)

test_strings = ["That is a happy dog","That is a very happy person","Today is a sunny day"]

embeddings = embeddings_model.embed_documents(test_strings)

print(embeddings)